In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import nltk
import re
import datasets
from tqdm import tqdm
import torch

from transformers import pipeline

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [14]:
classifier = pipeline("summarization", "Einmalumdiewelt/T5-Base_GNAD")


Downloading: 100%|██████████| 1.50k/1.50k [00:00<00:00, 1.50MB/s]
Downloading: 100%|██████████| 892M/892M [01:15<00:00, 11.8MB/s] 
Downloading: 100%|██████████| 2.37k/2.37k [00:00<00:00, 2.37MB/s]
Downloading: 100%|██████████| 792k/792k [00:00<00:00, 8.11MB/s]
Downloading: 100%|██████████| 2.42M/2.42M [00:02<00:00, 942kB/s] 
Downloading: 100%|██████████| 2.20k/2.20k [00:00<00:00, 2.21MB/s]


In [18]:
with open('input/AFD_Wahlprogramm_2021.txt') as f:
    lines = f.readlines()
lines

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 1469: character maps to <undefined>